In [ ]:
import pandas as pd
import numpy as np
from configparser import ConfigParser
import datetime as dt
from os.path import exists

config_path = 'config.ini'
config = ConfigParser()
config.read(config_path)

Month = config['Parameters']['Month']
Year = config['Parameters']['Year']
Month_Name = dt.datetime.strptime(str(Month), '%m').strftime('%B')

process = 'Medical Records'

inputs = (config[process]['InputFileOutputPath'] + config[process]['ETMOutputNamingConvention']).replace('MMMM', Month_Name).replace('MM', Month).replace('YYYY', Year)
outputs = (config[process]['OutputFileOutputPath'] + config[process]['OutputFileNamingConvention']).replace('MMMM',
                                                                                                       Month_Name).replace('MM', Month).replace('YYYY', Year)
shs = config[process]['SHSFullFile'].replace('MMMM',Month_Name).replace('MM', Month).replace('YYYY', Year)
pics = config[process]['PICfile'].replace('MMMM', Month_Name).replace('MM', Month).replace('YYYY', Year)

if exists('test.xlsx'):
    df = pd.read_excel('test.xlsx', engine = 'openpyxl')
else:
    inputs = pd.read_excel(inputs, engine='openpyxl')
    outputs = pd.read_excel(outputs, engine='openpyxl')
    shs = pd.read_excel(shs, engine = 'openpyxl', usecols=['INVNUM', 'Reason', 'BotName', 'RetrievalStatus', 'RetrievalDescription','BOTRequestDate', 'LastModifiedDate'])
    pics = pd.read_csv(pics, sep = '\t', header=None)

    bot_names = config[process]['SHSBotName'].split(',')
    corr_pics = config[process]['CorrPICs'].split(',')
    shs = shs[shs['BotName'].isin(bot_names)]
    shs['BOTRequestDate'] = pd.to_datetime(shs['BOTRequestDate'])
    shs['BOTRequestDate'] = pd.to_datetime(shs['BOTRequestDate']).dt.date
    shs['LastModifiedDate'] = pd.to_datetime(shs['LastModifiedDate'])
    shs['LastModifiedDate'] = pd.to_datetime(shs['LastModifiedDate']).dt.date


    pics.drop(columns=[1,2,3,4], inplace=True)
    pics.columns = ['Invoice', 'Post Date', 'Code']
    pics['Post Date'] = pd.to_datetime(pics['Post Date'])
    pic_desc = pd.read_excel('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/Dictionary 6 - RPA Only '
                             'Rejections.xlsx', engine='openpyxl')
    pics = pics[pics['Code'].isin(corr_pics)]
    pics = pics.merge(pic_desc, how='left')
    conditions = [
        inputs['Process'] == 'AllScripts',
        inputs['Process'] == 'O&C',
        inputs['Process'] =='Powersign',
        inputs['Process'] == 'Sunrise'
    ]

    inputs['SHS Name'] = np.select(conditions, bot_names)
    shs['LastModifiedDate'] = pd.to_datetime(shs['LastModifiedDate'])
    shs['BOTRequestDate'] = pd.to_datetime(shs['BOTRequestDate'])
    df = pd.merge(inputs, shs, how='left',
                  left_on = ['INVNUM', 'File Date', 'SHS Name'],
                  right_on=['INVNUM', 'BOTRequestDate', 'BotName'])
    df = pd.merge(df, pics, how='left',
                  left_on = ['INVNUM'],
                  right_on=['Invoice'])
    outputs['Process'] = outputs['Process'].str.strip()
    df = pd.merge(df, outputs, how='left',
                  left_on = ['INVNUM', 'File Date', 'Process'],
                  right_on = ['Invoice', 'File Date', 'Process'])
    cond = [(df['Code'].isnull()),
            (~(df['Code'].isnull())) &
            (df['Post Date'] >= df['BOTRequestDate']) &
            (df['Post Date'] <= df['LastModifiedDate'])
            ]
    values = [True, True]
    df['Keep'] = np.select(cond, values, False)
    df.drop(columns=['SHS Name', 'Invoice_x', 'Invoice_y', 'Keep'], inplace=True)
    df.to_excel('test.xlsx')

In [3]:
data = df.copy()
df = data[['ACK', 'Form Letter', 'Records']]
df = df.fillna(0)
data = data.fillna('0').astype('str')

df['ACK'] = df['ACK'].astype('int64')
df['Form Letter'] = df['Form Letter'].astype('int64')
df['Records'] = df['Records'].astype('int64')

In [4]:
from sklearn.preprocessing import LabelEncoder
import joblib

le_payer = LabelEncoder()
le_process = LabelEncoder()
le_shs_status = LabelEncoder()
le_pic_desc = LabelEncoder()

le_status = LabelEncoder()
le_category = LabelEncoder()
le_comment = LabelEncoder()

le_payer.classes_ = np.load('../references/ML Training/classes/payer_classes.npy', allow_pickle=True)
le_process.classes_ = np.load('../references/ML Training/classes/process_classes.npy', allow_pickle=True)
le_shs_status.classes_ = np.load('../references/ML Training/classes/shs_status_classes.npy', allow_pickle=True)
le_pic_desc.classes_ = np.load('../references/ML Training/classes/pic_desc_classes.npy', allow_pickle=True)

le_status.classes_ = np.load('../references/ML Training/classes/status_classes.npy', allow_pickle=True)
le_category.classes_ = np.load('../references/ML Training/classes/category_classes.npy', allow_pickle=True)
le_comment.classes_ = np.load('../references/ML Training/classes/comment_classes.npy', allow_pickle=True)

In [5]:
df['payer_n'] = le_payer.transform(data['PAYER'])
df['process_n'] = le_process.transform(data['Process'])
df['shs_status_n'] = le_shs_status.transform(data['RetrievalStatus'])
df['pic_desc_n'] = le_pic_desc.transform(data['Name'])

In [6]:
copy = df.copy()

In [7]:
status = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/models/status_model.joblib')

In [8]:
df['status_n'] = status.predict(df)
data['Status'] = le_status.inverse_transform(df['status_n'])

In [9]:
copy = df.copy()

In [12]:
category = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/models/category_model.joblib')

In [13]:
df['category_n'] = category.predict(df)
data['Category'] = le_category.inverse_transform(df['category_n'])

In [14]:
comments = joblib.load('C:/Users/dpashayan/PycharmProjects/Monthly_Audit/references/ML Training/models/comment_model.joblib')

In [15]:
df['comment_n'] = comments.predict(df)
data['Comments'] = le_comment.inverse_transform(df['comment_n'])

In [16]:
"""
TODO:
INVNUM > Invoice
BOTRequestDate > SHS Start
LastModifiedDate > SHS End
RetrievalStatus > SHS Status
RetrievalDescription > Retrieval Description
BotName drop
Post Date > PIC Date
Code > PIC Code
Name > PIC Desc
Add Folder Date = SHS Start
Proof > Proofs
"""
data = data.rename(
    {'INVNUM': 'Invoice',
    'BOTRequestDate': 'SHS Start',
    'LastModifiedDate': 'SHS End',
    'RetrievalStatus': 'SHS Status',
    'RetrievalDescription': 'Retrieval Description',
    'Post Date': 'PIC Date',
    'Code' : 'PIC Code',
    'Name' : 'PIC Desc',
    'Proof': 'Proofs'}, axis = 'columns'
          )

In [17]:
data.drop(columns='BotName', axis='columns', inplace=True)

In [18]:
config_path = 'config.ini'
config = ConfigParser()
config.read(config_path)

Month = config['Parameters']['Month']
Year = config['Parameters']['Year']
Month_Name = dt.datetime.strptime(str(Month), '%m').strftime('%B')

In [19]:
output_path = (f'{config[process]["MonthlySummaryPath"]}/{Year} {Month} - Medical Records Summary.xlsx')
output_path = output_path.replace('MMMM',Month_Name).replace('MM', Month).replace('YYYY', Year)

In [20]:
data.to_excel(output_path, index=None)